Brandon Roemer and Vincent Lou (did I spell your name right?)

Initial model built from:
    https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
  

In [47]:
import math
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

#import sklearn
#import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [48]:
#Read In Data 

data = pd.read_csv('CleanData.csv') 
data.head()
y = data["Open"]
X = data.drop(['Open'], axis=1)
X.head()

,High,Low,Close,Volume,Open.1,High.1,Low.1,Close.1,Volume.1,Open.2,High.2,Low.2,Close.2,Volume.2,Open.3,High.3,Low.3,Close.3,Volume.3
0,0.207278,0.266271,0.207774,0.040335,0.317724,0.316066,0.314614,0.315107,0.014362,0.221744,0.221735,0.219592,0.218835,0.071630,0.121008,0.120398,0.120547,0.121613,0.265755
1,0.205954,0.268285,0.211104,0.020598,0.313595,0.312520,0.313730,0.316131,0.005885,0.216649,0.221677,0.222476,0.225798,0.064605,0.119878,0.121420,0.124930,0.127285,0.215039
2,0.204670,0.267314,0.208549,0.020888,0.310289,0.307719,0.308843,0.307302,0.008160,0.217424,0.217205,0.220296,0.218899,0.031678,0.124445,0.119637,0.127240,0.125837,0.218488
3,0.205098,0.267961,0.209943,0.032848,0.304350,0.303431,0.307455,0.307065,0.016842,0.216899,0.217857,0.221881,0.220296,0.035793,0.124038,0.120821,0.128372,0.126066,0.164931
4,0.205293,0.268213,0.209400,0.077209,0.305648,0.304895,0.307398,0.308051,0.003199,0.219603,0.218163,0.222712,0.221599,0.059641,0.124868,0.122011,0.128281,0.127560,0.189904


In [49]:
#Shift data to predict on next day
y = y.drop(y.index[0])
X = X.drop(X.index[len(X)-1])
print(X.shape)
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, random_state = 1234)

X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = np.reshape(X_train, [X_train.shape[0], 1, X_train.shape[1]])
X_test = np.reshape(X_test, [X_test.shape[0], 1,  X_test.shape[1]])
print(X_train.shape)

#TODO 
#Validation Set


(2832, 19)
(1897, 1, 19)


In [50]:
#Create Model 
model = Sequential()
model.add(LSTM(10, input_shape=(1, 19)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)


Epoch 1/100
 - 4s - loss: 0.0032
Epoch 2/100
 - 3s - loss: 1.9471e-04
Epoch 3/100
 - 3s - loss: 1.6135e-04
Epoch 4/100
 - 4s - loss: 1.3848e-04
Epoch 5/100
 - 4s - loss: 1.3378e-04
Epoch 6/100
 - 4s - loss: 1.1716e-04
Epoch 7/100
 - 4s - loss: 9.7332e-05
Epoch 8/100
 - 3s - loss: 8.9819e-05
Epoch 9/100
 - 3s - loss: 9.3237e-05
Epoch 10/100
 - 4s - loss: 7.5918e-05
Epoch 11/100
 - 4s - loss: 7.4625e-05
Epoch 12/100
 - 4s - loss: 8.6189e-05
Epoch 13/100
 - 3s - loss: 6.7792e-05
Epoch 14/100
 - 3s - loss: 6.6766e-05
Epoch 15/100
 - 3s - loss: 6.6149e-05
Epoch 16/100
 - 3s - loss: 6.8173e-05
Epoch 17/100
 - 3s - loss: 7.3509e-05
Epoch 18/100
 - 4s - loss: 6.4518e-05
Epoch 19/100
 - 4s - loss: 6.2023e-05
Epoch 20/100
 - 4s - loss: 6.1792e-05
Epoch 21/100
 - 3s - loss: 5.9469e-05
Epoch 22/100
 - 3s - loss: 5.8426e-05
Epoch 23/100
 - 3s - loss: 5.8559e-05
Epoch 24/100
 - 3s - loss: 7.5152e-05
Epoch 25/100
 - 5s - loss: 5.2766e-05
Epoch 26/100
 - 3s - loss: 5.5191e-05
Epoch 27/100
 - 4s - loss

In [64]:
from sklearn.preprocessing import MinMaxScaler

y_train = np.array(y_train)
y_test = np.array(y_test)

Human_Df = pd.read_csv("CleanDataHuman.csv")
Human_Df = Human_Df.drop(["Ticker"], axis = 1)

unTransformed_data = np.array(Human_Df)[4].reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit_transform(unTransformed_data )


# make predictions
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)
# invert predictions

trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([y_train])

testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([y_test])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))


Train Score: 1168.26 RMSE
Test Score: 1120.02 RMSE
